<a href="https://colab.research.google.com/github/Jayrbsn/image-classification-transfer-learning/blob/main/image_classification_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: imports and config
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

AUTOTUNE = tf.data.AUTOTUNE
IMG_SIZE = 224    # MobileNetV2 typical input
BATCH_SIZE = 32


In [ ]:
# Cell 2: load tf_flowers (80/20 split)
(ds_train, ds_val), ds_info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,   # returns (image, label)
    with_info=True
)

num_classes = ds_info.features['label'].num_classes
class_names = ds_info.features['label'].names

print("Num classes:", num_classes)
print("Class names:", class_names)


In [ ]:
# Cell 3: preprocessing + batching
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

def preprocess(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32)
    image = preprocess_input(image)   # scales to [-1, 1] for MobileNetV2
    return image, label

ds_train = ds_train.map(preprocess, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.shuffle(1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)

ds_val = ds_val.map(preprocess, num_parallel_calls=AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(AUTOTUNE)

print("Train batches:", tf.data.experimental.cardinality(ds_train).numpy())
print("Val batches:  ", tf.data.experimental.cardinality(ds_val).numpy())


In [ ]:
# Cell 4: plot a few training images (convert from [-1,1] to [0,1] for display)
plt.figure(figsize=(8,8))
for images, labels in ds_train.take(1):
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        img = images[i].numpy()
        img = (img + 1.0) / 2.0   # [-1,1] -> [0,1]
        plt.imshow(img)
        plt.title(class_names[int(labels[i])])
        plt.axis('off')
